# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,new norfolk,-42.7826,147.0587,13.17,81,99,0.76,AU,1681321742
1,1,albany,42.6001,-73.9662,24.09,32,37,6.28,US,1681321664
2,2,kapaa,22.0752,-159.3190,23.18,89,75,7.20,US,1681321743
3,3,tahoua,14.8888,5.2692,37.99,7,15,3.09,NE,1681321744
4,4,rikitea,-23.1203,-134.9692,25.23,69,17,5.58,PF,1681321744
5,5,kaitangata,-46.2817,169.8464,11.29,93,100,4.65,NZ,1681321744
6,6,punta arenas,-53.1500,-70.9167,6.06,56,0,3.60,CL,1681321368
7,7,east london,-33.0153,27.9116,18.52,82,0,1.54,ZA,1681321745
8,8,ushuaia,-54.8000,-68.3000,4.81,60,40,9.26,AR,1681321746
9,9,mataura,-46.1927,168.8643,11.38,98,100,2.68,NZ,1681321746


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.


In [4]:
%%capture --no-display
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    #scale = 0.01,
    color = "City"
)


# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [33]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_with_nulls_df = city_data_df.loc[(city_data_df['Max Temp']>=14) & (city_data_df['Max Temp']<25) & (city_data_df['Humidity']<78) & (city_data_df['Humidity']>70) & (city_data_df['Wind Speed']<3)]
                                

# Drop any rows with null values
ideal_weather_cities_df = ideal_weather_with_nulls_df.dropna()

# Display sample data
ideal_weather_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
52,52,siniscola,40.5719,9.6922,18.75,71,76,1.25,IT,1681321768
90,90,bredasdorp,-34.5322,20.0403,14.54,71,98,2.63,ZA,1681321782
97,97,poli,-3.3333,36.8000,21.15,73,40,2.57,TZ,1681321784
181,181,pascagoula,30.3658,-88.5561,21.40,74,100,2.57,US,1681321814
265,265,esperance,-33.8667,121.9000,17.73,71,0,2.54,AU,1681321844
270,270,praia da vitoria,38.7333,-27.0667,18.14,77,40,1.58,PT,1681321846
346,346,chepareria,1.3077,35.2039,18.20,72,94,2.33,KE,1681321868
381,381,coquimbo,-29.9533,-71.3436,18.85,72,0,2.57,CL,1681321690
534,534,mumbwa,-14.9783,27.0619,20.15,77,100,2.54,ZM,1681321920
539,539,basoko,1.2391,23.6160,24.25,75,100,1.90,CD,1681321921


### Step 3: Create a new DataFrame called `hotel_df`.

In [34]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_cities_df[['City','Country','Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
52,siniscola,IT,40.5719,9.6922,71,
90,bredasdorp,ZA,-34.5322,20.0403,71,
97,poli,TZ,-3.3333,36.8000,73,
181,pascagoula,US,30.3658,-88.5561,74,
265,esperance,AU,-33.8667,121.9000,71,
270,praia da vitoria,PT,38.7333,-27.0667,77,
346,chepareria,KE,1.3077,35.2039,72,
381,coquimbo,CL,-29.9533,-71.3436,72,
534,mumbwa,ZM,-14.9783,27.0619,77,
539,basoko,CD,1.2391,23.6160,75,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [35]:
# Set parameters to search for a hotel
categories = "accommodation.hotel"
radius = 10000
limit = 20

# These parameters change for each city
#filters = f"circle:{longitude},{latitude},{radius}"
#bias = f"proximity:{longitude},{latitude}"

params = {
    "categories":categories,
    "limit":limit,
    "apiKey":geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    current_lat = hotel_df.loc[index, 'Lat']
    current_lng = hotel_df.loc[index, 'Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{current_lng},{current_lat},{radius}"
    params["bias"] = f"proximity:{current_lng},{current_lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    #print(f"URL = {name_address.url}")
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        #print(f"for city -  {name_address["features"][0]["properties"]["name"]}")
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
siniscola - nearest hotel: Agriturismo Sa Costa
bredasdorp - nearest hotel: Victoria Hotel
poli - nearest hotel: Imful
pascagoula - nearest hotel: Grand Magnolia
esperance - nearest hotel: Hospitality
praia da vitoria - nearest hotel: Salles
chepareria - nearest hotel: No hotel found
coquimbo - nearest hotel: Hotel Iberia
mumbwa - nearest hotel: La Hacienda
basoko - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
52,siniscola,IT,40.5719,9.6922,71,Agriturismo Sa Costa
90,bredasdorp,ZA,-34.5322,20.0403,71,Victoria Hotel
97,poli,TZ,-3.3333,36.8000,73,Imful
181,pascagoula,US,30.3658,-88.5561,74,Grand Magnolia
265,esperance,AU,-33.8667,121.9000,71,Hospitality
270,praia da vitoria,PT,38.7333,-27.0667,77,Salles
346,chepareria,KE,1.3077,35.2039,72,No hotel found
381,coquimbo,CL,-29.9533,-71.3436,72,Hotel Iberia
534,mumbwa,ZM,-14.9783,27.0619,77,La Hacienda
539,basoko,CD,1.2391,23.6160,75,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [36]:
%%capture --no-display

# Configure the map plot
map_plot_1 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    #scale = 0.01,
    color = "City",
    hover_cols = (["Hotel Name","Country"])
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)